<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, 
and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - 
a function that can take, as an argument, the size of text (e.g. number of characters or lines) 
to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, 
to sample/use smaller data and parameters, so you can have a tighter feedback 
loop when you're trying to get things running. Then, once you've got a proof of concept - 
start pushing it more!

In [ ]:
import Tokenizer
abstracts[100][:300]
#create tokenizer object
tokenizer = Tokenizer(num_words = None, filters = '!"#$%&()/+*-,./:;<=>?@[\\]^_{|}~\t\n', lower=True, split='')
#train tokenizer ot the texts
tokenizer.fit_on_texts(abstracts)
#conert list of string into list of list of integer
sequences = tokenizer.texts_to_sequence(abstracts)
sequences[100][:15]
#mapping of indexes to words
idx_word=tokenizer.index_word
' '.join(idx_word[w] for w in sequences[100][:40])

In [ ]:
import numpy as np
features =[]
labels = []
training_length = 50
#iterate through the sequences of tokens
for seq in sequences:
    #create multiple training example frome ach seqences
    for i in range(training_length, len(seq)):
        #extract the features and label
        extract = seq[i - training_length:i +1]
        #set the feature and label
        features.append(extract[:-1])
        labels.append(extract[-1])
features = np.array(features)

In [ ]:
num_words = len(word_idx) + 1
#empty array to hold labels
label_array  =np.zeros((len(features), num_words), dtype=np.int8)
#1 hot encode the label
for examle_index, word_index in enumerate(labels):
    label_array[example_index, word_index] = 1
label_array.shape

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

model = Sequential()

#embed layer
model.add(
    Embedding(input_dim=num_words,
              input_length = training_length,
              output_dim=100,
              weights=[embedding_matrix],
              trainable=False,
              mask_zero=True)
)
#masking layer for pre_train
model.add(Masking(mask_value=0.0))

#recurrent layer
model.add(LSTM(64, return_sequence=False, dropout=.1, recurrent_dropout=.1))

#fully connected layer
model.add(Dense(64, activation='relu'))

#dropout for regularization
model.add(Dropout(.5))

#output layer
model.add(Dense(num_words, activation='softmax'))
#compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics =['accuracy'])

In [ ]:
glove_vector = 'https://www.gutenberg.org/files/100/100-0.txt'
glove = np.loadtxt(glove_vector, dtype='str', comments = None)
#extract the vector and words
vectors = glove[:,1:].astype('float')
words=glove[:,0]
#create lookup of words to vectors
word_lookup = {word:vector for word, vector in zip(words,vectors)}
#new matrix to hold word embedding
embedding_matrix = np.zeros((num_words, vector.shape[1]))
for i, word in enumerate(word_idx.keys()):
    #look up word embedding
    vector = word_lookup.get(word,None)
    
    #record in matrix
    if vector is not None:
        embedding_matrix[i+1, :] = vector
        

In [ ]:
word_lookup['neural'][:10]

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
#create callbacks
callbacks = [EarlyStopping(monitor='val_loss', patience=5), ModelCheckpoint('../models/model.h5'), save_best_only=True, save_weights_only=False]

In [ ]:
history = model.fit(X_train, y_train, batch_size=2048, epochs=150, callbacks=callbacks, validation_data=(X_valid,y_valid))

In [ ]:
from keras import load_model
model = load_model('../models/model.h5')
model.evaluate(X_valid,y_valid)

In [3]:
import numpy as np
class RNN:
    #
def step(self, x):
    #update the hidden state
    #W_hh matrix
    #W_xh matrix
    #w_hy matrix
    #tanh - based previous hidden state, one is current input. keep between [-1,1]
    self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))
    #compute the output vector
    y = np.dot(self.Why, self.h)
    return y

IndentationError: expected an indented block (<ipython-input-3-7bde015050c1>, line 4)

In [4]:
rnn = RNN()
y = rnn.step(x) # x is an input vector, y is the RNN output vector
# TODO - Words, words, mere words, no matter from the heart.
y1 = rnn2

NameError: name 'RNN' is not defined

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN